In [1]:
import json
from pathlib import Path
import subprocess
import logging
import copy
import os
import shutil  # For copying files
from minio import Minio
import ray
import hashlib
import datetime

/home/antoine/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-01-13 16:33:22,012	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
# Initialize Ray cluster (adjust parameters for your setup)
ray.init(ignore_reinit_error=True)

2025-01-13 16:33:24,752	INFO worker.py:1812 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


Python version:,3.12.3
Ray version:,2.40.0
Dashboard:,http://127.0.0.1:8265


In [3]:
# Configure logging
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s [%(levelname)s] %(message)s"
)

In [4]:
# Define paths and parameters
original_config_path = Path("config/export.json").resolve()
polyfem_bin = Path("/home/antoine/code/polyfem/build/PolyFEM_bin").resolve()
base_output = Path("output").resolve()
# Ensure base output directory exists
base_output.mkdir(parents=True, exist_ok=True)

In [5]:
# Log paths
logging.info(f"Current Working Directory: {os.getcwd()}")
logging.info(f"Original Config Path: {original_config_path}")
logging.info(f"PolyFEM Binary Path: {polyfem_bin}")
logging.info(f"Base Output Directory: {base_output}")


2025-01-13 16:33:26,094 [INFO] Current Working Directory: /home/antoine/code/BatchPolyfem/notebooks
2025-01-13 16:33:26,098 [INFO] Original Config Path: /home/antoine/code/BatchPolyfem/notebooks/config/export.json
2025-01-13 16:33:26,100 [INFO] PolyFEM Binary Path: /home/antoine/code/polyfem/build/PolyFEM_bin
2025-01-13 16:33:26,100 [INFO] Base Output Directory: /home/antoine/code/BatchPolyfem/notebooks/output


In [6]:
new_materials_list = [
    # Material pairs with their properties from the table
    [
        {"id": 1, "type": "NeoHookean", "E": 200e9, "nu": 0.25, "rho": 8000},  # Steel
        {"id": 2, "type": "NeoHookean", "E": 200e9, "nu": 0.25, "rho": 8000},
    ],  # Steel
    [
        {"id": 1, "type": "NeoHookean", "E": 133e9, "nu": 0.35, "rho": 8940},  # Copper
        {"id": 2, "type": "NeoHookean", "E": 133e9, "nu": 0.35, "rho": 8940},
    ],  # Copper
    [
        {"id": 1, "type": "NeoHookean", "E": 69e9, "nu": 0.33, "rho": 2700},  # Aluminum
        {"id": 2, "type": "NeoHookean", "E": 69e9, "nu": 0.33, "rho": 2700},
    ],  # Aluminum
    [
        {"id": 1, "type": "NeoHookean", "E": 69e9, "nu": 0.33, "rho": 2700},  # Aluminum
        {"id": 2, "type": "NeoHookean", "E": 200e9, "nu": 0.25, "rho": 8000},
    ],  # Steel
    [
        {"id": 1, "type": "NeoHookean", "E": 115e9, "nu": 0.34, "rho": 8730},  # Brass
        {"id": 2, "type": "NeoHookean", "E": 170e9, "nu": 0.26, "rho": 7200},
    ],  # Cast Iron
    [
        {"id": 1, "type": "NeoHookean", "E": 64e9, "nu": 0.31, "rho": 8650},  # Cadmium
        {"id": 2, "type": "NeoHookean", "E": 64e9, "nu": 0.31, "rho": 8650},
    ],  # Cadmium
    [
        {"id": 1, "type": "NeoHookean", "E": 64e9, "nu": 0.31, "rho": 8650},  # Cadmium
        {"id": 2, "type": "NeoHookean", "E": 200e9, "nu": 0.25, "rho": 8000},
    ],  # Steel
    [
        {
            "id": 1,
            "type": "NeoHookean",
            "E": 170e9,
            "nu": 0.26,
            "rho": 7200,
        },  # Cast Iron
        {"id": 2, "type": "NeoHookean", "E": 170e9, "nu": 0.26, "rho": 7200},
    ],  # Cast Iron
    [
        {
            "id": 1,
            "type": "NeoHookean",
            "E": 248e9,
            "nu": 0.31,
            "rho": 7190,
        },  # Chromium
        {"id": 2, "type": "NeoHookean", "E": 248e9, "nu": 0.31, "rho": 7190},
    ],  # Chromium
    [
        {"id": 1, "type": "NeoHookean", "E": 133e9, "nu": 0.35, "rho": 8940},  # Copper
        {"id": 2, "type": "NeoHookean", "E": 170e9, "nu": 0.26, "rho": 7200},
    ],  # Cast Iron
    [
        {"id": 1, "type": "NeoHookean", "E": 133e9, "nu": 0.35, "rho": 8940},  # Copper
        {"id": 2, "type": "NeoHookean", "E": 133e9, "nu": 0.35, "rho": 8940},
    ],  # Copper
    [
        {"id": 1, "type": "NeoHookean", "E": 133e9, "nu": 0.35, "rho": 8940},  # Copper
        {"id": 2, "type": "NeoHookean", "E": 200e9, "nu": 0.25, "rho": 8000},
    ],  # Steel
    [
        {"id": 1, "type": "NeoHookean", "E": 60e9, "nu": 0.25, "rho": 2400},  # Glass
        {"id": 2, "type": "NeoHookean", "E": 60e9, "nu": 0.25, "rho": 2400},
    ],  # Glass
    [
        {"id": 1, "type": "NeoHookean", "E": 60e9, "nu": 0.25, "rho": 2400},  # Glass
        {"id": 2, "type": "NeoHookean", "E": 200e9, "nu": 0.25, "rho": 8000},
    ],  # Steel
    [
        {"id": 1, "type": "NeoHookean", "E": 60e9, "nu": 0.25, "rho": 2400},  # Glass
        {"id": 2, "type": "NeoHookean", "E": 170e9, "nu": 0.31, "rho": 8900},
    ],  # Nickel
    [
        {"id": 1, "type": "NeoHookean", "E": 20e9, "nu": 0.2, "rho": 2050},  # Graphite
        {"id": 2, "type": "NeoHookean", "E": 20e9, "nu": 0.2, "rho": 2050},
    ],  # Graphite
    [
        {"id": 1, "type": "NeoHookean", "E": 20e9, "nu": 0.2, "rho": 2050},  # Graphite
        {"id": 2, "type": "NeoHookean", "E": 200e9, "nu": 0.25, "rho": 8000},
    ],  # Steel
    [
        {"id": 1, "type": "NeoHookean", "E": 170e9, "nu": 0.31, "rho": 8900},  # Nickel
        {"id": 2, "type": "NeoHookean", "E": 170e9, "nu": 0.31, "rho": 8900},
    ],  # Nickel
    [
        {"id": 1, "type": "NeoHookean", "E": 170e9, "nu": 0.31, "rho": 8900},  # Nickel
        {"id": 2, "type": "NeoHookean", "E": 200e9, "nu": 0.25, "rho": 8000},
    ],  # Steel
    [
        {"id": 1, "type": "NeoHookean", "E": 4e9, "nu": 0.39, "rho": 1130},  # Nylon
        {"id": 2, "type": "NeoHookean", "E": 4e9, "nu": 0.39, "rho": 1130},
    ],  # Nylon
    [
        {
            "id": 1,
            "type": "NeoHookean",
            "E": 3.3e9,
            "nu": 0.37,
            "rho": 1190,
        },  # Plexiglass
        {"id": 2, "type": "NeoHookean", "E": 3.3e9, "nu": 0.37, "rho": 1190},
    ],  # Plexiglass
    [
        {
            "id": 1,
            "type": "NeoHookean",
            "E": 3.3e9,
            "nu": 0.37,
            "rho": 1190,
        },  # Plexiglass
        {"id": 2, "type": "NeoHookean", "E": 200e9, "nu": 0.25, "rho": 8000},
    ],  # Steel
    [
        {
            "id": 1,
            "type": "NeoHookean",
            "E": 2.5e9,
            "nu": 0.4,
            "rho": 1040,
        },  # Polystyrene
        {"id": 2, "type": "NeoHookean", "E": 2.5e9, "nu": 0.4, "rho": 1040},
    ],  # Polystyrene
    [
        {
            "id": 1,
            "type": "NeoHookean",
            "E": 2.5e9,
            "nu": 0.4,
            "rho": 1040,
        },  # Polystyrene
        {"id": 2, "type": "NeoHookean", "E": 200e9, "nu": 0.25, "rho": 8000},
    ],  # Steel
    [
        {"id": 1, "type": "NeoHookean", "E": 0.01e9, "nu": 0.47, "rho": 2300},  # Rubber
        {"id": 2, "type": "NeoHookean", "E": 3e9, "nu": 0.35, "rho": 2500},
    ],  # Asphalt
    [
        {"id": 1, "type": "NeoHookean", "E": 115e9, "nu": 0.34, "rho": 8730},  # Brass
        {"id": 2, "type": "NeoHookean", "E": 200e9, "nu": 0.25, "rho": 8000},
    ],  # Steel
    [
        {"id": 1, "type": "NeoHookean", "E": 200e9, "nu": 0.25, "rho": 8000},  # Steel
        {"id": 2, "type": "NeoHookean", "E": 170e9, "nu": 0.26, "rho": 7200},
    ],  # Cast Iron
    [
        {"id": 1, "type": "NeoHookean", "E": 0.5e9, "nu": 0.47, "rho": 2200},  # Teflon
        {"id": 2, "type": "NeoHookean", "E": 0.5e9, "nu": 0.47, "rho": 2200},
    ],  # Teflon
    [
        {"id": 1, "type": "NeoHookean", "E": 10e9, "nu": 0.35, "rho": 750},  # Wood
        {"id": 2, "type": "NeoHookean", "E": 10e9, "nu": 0.35, "rho": 750},
    ],  # Wood
    [
        {"id": 1, "type": "NeoHookean", "E": 82.7e9, "nu": 0.25, "rho": 7120},  # Zinc
        {"id": 2, "type": "NeoHookean", "E": 170e9, "nu": 0.26, "rho": 7200},
    ],  # Cast Iron
]


In [7]:
mesh_0 = [
    "cube.msh",
    "cube2.msh",
]

mesh_1 = [
    "cube.msh",
    "cube2.msh",
    "sine_amplitude-50.0_frequency-200.0_hmm_base-1_error-0.001_z_exagg-1.0_z_value-5.0_ftetwild_.msh",
]

In [8]:
def read_json_config(file_path: Path) -> dict:
    with file_path.open("r") as f:
        return json.load(f)

In [9]:
def read_full_config(config_path: Path) -> dict:
    config = read_json_config(config_path)
    if "common" in config:
        common_path = Path(config_path.parent) / config["common"]
        common_config = read_json_config(common_path)
        merged_config = copy.deepcopy(common_config)
        for key, value in config.items():
            if key != "common":
                merged_config[key] = value
        return merged_config
    return config

In [10]:
def validate_config(config: dict):
    if "materials" not in config or not isinstance(config["materials"], list):
        raise ValueError("Config must contain a 'materials' list.")

In [11]:
def save_modified_config(config: dict, output_folder: Path, base_name: str) -> Path:
    new_file_path = output_folder / f"{base_name}_modified.json"
    with new_file_path.open("w") as f:
        json.dump(config, f, indent=2)
    return new_file_path

In [12]:
def find_mesh_files_in_config(config: dict) -> list:
    mesh_files = []
    geometry_items = config.get("geometry", [])
    for item in geometry_items:
        if "mesh" in item:
            mesh_files.append(item["mesh"])
    return mesh_files

In [13]:
def copy_mesh_files(original_config_path, destination_folder: Path, mesh_files: list):
    for mesh_file in mesh_files:
        source = Path(original_config_path.parent) / mesh_file
        if source.exists():
            shutil.copy(source, destination_folder)
            logging.info(f"Copied {mesh_file} to {destination_folder}")
        else:
            logging.warning(f"Mesh file {mesh_file} not found at {source}.")

In [14]:
def run_polyfem(json_file: Path, output_folder: Path, polyfem_bin: Path):
    # Check if the output folder exists and 20 files are present
    if output_folder.exists() and len(list(output_folder.glob("*.vtu"))) == 20:
        logging.info(
            f"Skipping simulation for {json_file}, output folder already exists: {output_folder}"
        )
        return

    # Create the output folder if it doesn't exist
    output_folder.mkdir(parents=True, exist_ok=True)

    cmd = [str(polyfem_bin), "--json", str(json_file), "-o", str(output_folder)]
    logging.info(f"Executing PolyFEM simulation for {json_file}")

    try:
        # Run the subprocess and suppress stdout and stderr
        subprocess.run(
            cmd, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL, check=True
        )
        logging.info(f"PolyFEM simulation completed for: {json_file}")
    except subprocess.CalledProcessError as e:
        logging.error(f"PolyFEM simulation failed for {json_file}: {e}")

In [15]:
def modify_config_with_new_materials(
    original_config: Path, new_materials: list, output_folder: Path
) -> Path:
    config = read_full_config(original_config)
    config["materials"] = new_materials
    validate_config(config)
    return save_modified_config(config, output_folder, original_config.stem)

In [16]:
def generate_descriptive_folder_name(
    simulation_index: int, materials: list, mesh_files: list
) -> str:
    """
    Generate a descriptive folder name using simulation index, material properties, and mesh file names.
    """
    # Use a hash of material properties for brevity
    material_str = "_".join(
        f"E{mat['E']}_nu{mat['nu']}_rho{mat['rho']}" for mat in materials
    )
    material_hash = hashlib.md5(material_str.encode()).hexdigest()[:8]
    # Include mesh file names (or part of them) in folder name
    mesh_part = "_".join(mesh for mesh in mesh_files) if mesh_files else "no_mesh"

    folder_name = f"sim_{material_str}_{material_hash}_{mesh_part}"
    return folder_name


In [17]:
@ray.remote
def run_simulation_for_materials(materials: list, simulation_index: int) -> str:
    """
    Ray remote function to run a simulation.
    Returns the simulation directory path on success.
    """
    try:
        # Read full configuration once to extract mesh files for naming
        full_config = read_full_config(original_config_path)
        mesh_files_in_config = find_mesh_files_in_config(full_config)

        # Generate a descriptive folder name
        descriptive_folder = generate_descriptive_folder_name(
            simulation_index, materials, mesh_files_in_config
        )
        simulation_output_dir = base_output / descriptive_folder
        simulation_output_dir.mkdir(parents=True, exist_ok=True)

        logging.info(
            f"Simulation {simulation_index} Output Directory: {simulation_output_dir}"
        )

        modified_config_path = modify_config_with_new_materials(
            original_config_path, materials, simulation_output_dir
        )
        logging.info(
            f"Modified Config Path for Simulation {simulation_index}: {modified_config_path}"
        )

        copy_mesh_files(
            original_config_path, simulation_output_dir, mesh_files_in_config
        )

        run_polyfem(modified_config_path, simulation_output_dir, polyfem_bin)

        logging.info(f"Simulation {simulation_index} completed successfully.")
        return str(simulation_output_dir)
    except Exception as e:
        logging.error(f"Simulation {simulation_index} failed: {e}")
        return ""

In [18]:
# Dispatch Ray tasks for each simulation
ray_futures = []
for idx, materials in enumerate(new_materials_list):
    future = run_simulation_for_materials.remote(materials, idx)
    ray_futures.append(future)

# Wait for all simulations to complete and get their output directories
simulation_dirs = ray.get(ray_futures)

In [ ]:
# Initialize MinIO client
minio_client = Minio(
    "minio.antoineboucher.info",
    access_key="minioadmin",
    secret_key="minioadmin",
    secure=True,
)
bucket_name = "testing-bucket"
try:
    if not minio_client.bucket_exists(bucket_name):
        minio_client.make_bucket(bucket_name)
        logging.info(f"Bucket '{bucket_name}' created.")
    else:
        logging.info(f"Bucket '{bucket_name}' already exists.")
except Exception as e:
    logging.error(f"Bucket check/creation failed: {e}")


def upload_directory_to_minio(
    client: Minio, bucket: str, source_dir: Path, dest_prefix: str = ""
):
    for root, _, files in os.walk(source_dir):
        for file in files:
            file_path = Path(root) / file
            # Create object name relative to the base_output directory
            relative_path = file_path.relative_to(base_output)
            object_name = f"{dest_prefix}/{relative_path}".lstrip("/")
            try:
                client.fput_object(bucket, object_name, str(file_path))
                # logging.info(f"Uploaded {file_path} to {bucket}/{object_name}")
            except Exception as e:
                logging.error(f"Failed to upload {file_path}: {e}")


# Upload simulation directories to MinIO
for simulation_dir in simulation_dirs:
    upload_directory_to_minio(minio_client, bucket_name, Path(simulation_dir))
    # delete the directory locally after uploading
    shutil.rmtree(simulation_dir)


# Shutdown Ray when done
ray.shutdown()